In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seed = 6180
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.7, seed=seed)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions


In [10]:
import os

ofp = "../models_temp/Similarity_Hybrid/seed6180/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 3749.2876352484},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_all' : {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'SLIM_all2' : {'topK': 4997, 'l1_ratio': 0.00036065147205858187, 'alpha': 0.03203200564199538, 'mw': 1.0785132020354402, 'workers': 8},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 295, "learning_rate": 9.598708662230564e-05, "l2_reg": 1.4122381520119457e-05, "dropout": 0.5759445339123964, "total_anneal_steps": 147387, "anneal_cap": 0.4376000433422989, "batch_size": 256, "encoding_size": 185, "next_layer_size_multiplier": 6, "max_n_hidden_layers": 1, "max_layer_size": 5000.0},
                            'EASE_stk': {"topK": None, "normalize_matrix": False, 'mw': 4.0, "l2_norm": 3749.2876352484},
                         }


In [5]:
def model_init(recommender, name, args, seed):
    path_name = ofp + name + str(seed) + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name + str(seed))
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name + str(seed))

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender_Hybrid

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [8]:
groups = ld.load_groups(5)

In [11]:
recommender_slim_all = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_all2 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_wei = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_weighted)
recommender_ease = EASE_R_Recommender(URM_train)
recommender_itemknn = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender_rp3 = RP3betaRecommender(URM_train)
#recommender_svd = PureSVDRecommender(URM_train)
recommender_ease_hyb = EASE_R_Recommender_Hybrid(URM_train, ICM_all)

model_init(recommender_slim_all, 'SLIMall', models_to_combine_best['SLIM_all'], seed)
model_init(recommender_slim_all2, 'SLIMall2', models_to_combine_best['SLIM_all2'], seed)
model_init(recommender_slim_wei, 'SLIMweig', models_to_combine_best['SLIMweig'], seed)
model_init(recommender_ease, 'EASE', models_to_combine_best['EASE'], seed)
model_init(recommender_rp3, 'RP3beta', models_to_combine_best['RP3beta'], seed)
model_init(recommender_itemknn, 'ICM_all', models_to_combine_best['icm_all'], seed)
#model_init(recommender_svd, 'PureSVD6180', {'num_factors': 30})
model_init(recommender_ease_hyb, 'EASE_stacked', models_to_combine_best['EASE_stk'], seed)

recommender_slim_all.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_all2.RECOMMENDER_NAME = 'SLIMall2'
recommender_slim_wei.RECOMMENDER_NAME = 'SLIMweig'
recommender_ease.RECOMMENDER_NAME = 'EASE'
recommender_itemknn.RECOMMENDER_NAME = 'KNN'
recommender_rp3.RECOMMENDER_NAME = 'RP3'
#recommender_svd.RECOMMENDER_NAME = 'SVD'
recommender_ease_hyb.RECOMMENDER_NAME = 'EASE_Hybrid'

../models_temp/Similarity_Hybrid/seed6180/SLIMall6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMall6180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMall26180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMall26180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMweig6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMweig6180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/EASE6180.zip
Model found!
EASE_R_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/EASE6180'
EASE_R_Recommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/RP3beta6180.zip
Model found!
RP3betaRecommender: Loading model from file '../models_te

In [12]:
recommender_slim_bpr = SLIM_BPR_Cython(URM_train)
recommender_slim_bpr_hyb = SLIM_BPR_Cython_Hybrid(URM_train, ICM_weighted)
recommender_ials_hyb = IALSRecommender_Hybrid(URM_train, ICM_weighted)
#recommender_p3alpha = P3alphaRecommender(URM_train)
recommender_slimer = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

model_init(recommender_slim_bpr, 'SLIMBPR', models_to_combine_best['SLIMBPR'], seed)
model_init(recommender_slim_bpr_hyb, 'SLIMBPR_Hyb', models_to_combine_best['SLIM_BPR_Hyb'], seed)
model_init(recommender_ials_hyb, 'IALS_Hyb', models_to_combine_best['IALS_Hyb'], seed)
#model_init(recommender_p3alpha, 'P3alpha6180', models_to_combine_best['P3alpha'])
model_init(recommender_slimer, 'SLIMER', models_to_combine_best['SLIMER'], seed)

recommender_slim_bpr.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_bpr_hyb.RECOMMENDER_NAME = 'SLIMall2'
recommender_ials_hyb.RECOMMENDER_NAME = 'IALS_Hyb'
#recommender_p3alpha.RECOMMENDER_NAME = 'P3alpha'
recommender_slimer.RECOMMENDER_NAME = 'SLIMER'

../models_temp/Similarity_Hybrid/seed6180/SLIMBPR6180.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMBPR6180'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMBPR_Hyb6180.zip
Model found!
SLIM_BPR_Recommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMBPR_Hyb6180'
SLIM_BPR_Recommender_Hybrid: Loading complete
../models_temp/Similarity_Hybrid/seed6180/IALS_Hyb6180.zip
Model found!
IALSRecommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/IALS_Hyb6180'
IALSRecommender_Hybrid: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMER6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMER6180'
SLIMElasticNetRecommender: Loading complete


In [ ]:
all_rec = [recommender_slim_all, recommender_slim_all2, recommender_slim_wei, recommender_ease, recommender_itemknn, recommender_rp3, recommender_svd, recommender_slim_bpr, recommender_slim_bpr_hyb, recommender_ials_hyb, recommender_slimer, recommender_slim_all2_ease]
dict_res = rank_models(evaluator_validation, all_rec)
dict_res2 = rank_models_group_wise(evaluators, all_rec)

In [ ]:
for i in dict_res.keys():
    dict_res[i] = {k: v for k, v in sorted(dict_res[i].items(), key=lambda item: item[1])}
    print(i, dict_res[i])
    
dict_res2 = {k: v for k, v in sorted(dict_res2.items(), key=lambda item: item[1])}
print(dict_res2)

In [13]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        w_1 = trial.suggest_uniform('w_1', 0, 1)
        w_2 = trial.suggest_uniform('w_2', 0, 1)
        w_3 = trial.suggest_uniform('w_3', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(None, w_1, w_2, w_3)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            trial.report(map_v, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender_slimer, recommender_ease_hyb, recommender_rp3, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-07 15:57:23,906] A new study created in memory with name: no-name-c69ca0d1-a13a-4e39-a9f2-2617cf5d70e4


CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.49210548975496793
EASE_Hybrid with weight beta: 0.27718074879531224
RP3 with weight gamma: 0.23071376144971983
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 51.22 sec. Users per second: 267


[I 2022-01-07 15:58:15,212] Trial 0 finished with value: -0.3446512157992881 and parameters: {'w_1': 0.7928983691721041, 'w_2': 0.4466037633417541, 'w_3': 0.37173445329807553}. Best is trial 0 with value: -0.3446512157992881.


             MAP
cutoff          
10      0.344651
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.33378432462105445
EASE_Hybrid with weight beta: 0.5429413568928073
RP3 with weight gamma: 0.1232743184861382
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 51.53 sec. Users per second: 265


[I 2022-01-07 15:59:06,827] Trial 1 finished with value: -0.34403365791086643 and parameters: {'w_1': 0.47392389353026243, 'w_2': 0.7708956437944487, 'w_3': 0.17503112243382757}. Best is trial 0 with value: -0.3446512157992881.


...

             MAP
cutoff          
10      0.344914
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.4828605454789972
EASE_Hybrid with weight beta: 0.23955054274148846
RP3 with weight gamma: 0.2775889117795143
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 1.54 min. Users per second: 148


[I 2022-01-07 19:17:09,164] Trial 135 finished with value: -0.3447865329979436 and parameters: {'w_1': 0.9762239049341435, 'w_2': 0.4843116060190948, 'w_3': 0.5612157256604929}. Best is trial 121 with value: -0.3449817010839833.


             MAP
cutoff          
10      0.344787
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.46443641959824306
EASE_Hybrid with weight beta: 0.21784465057084243
RP3 with weight gamma: 0.31771892983091443
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 1.67 min. Users per second: 136


[I 2022-01-07 19:18:49,485] Trial 136 finished with value: -0.3447738569137599 and parameters: {'w_1': 0.9461448973434362, 'w_2': 0.44379078783154535, 'w_3': 0.6472535993386901}. Best is trial 121 with value: -0.3449817010839833.


             MAP
cutoff          
10      0.344774
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.486506787497041
EASE_Hybrid with weight beta: 0.22898477136034534
RP3 with weight gamma: 0.2845084411426136
Norm type: None
